# Custom Filters
In addition to the filters documented in Google Charts, JoogleCharts provides some custom filters.  The custom filters are typically extensions of the filters provided by Google Charts, so be sure to read about how to use the regular filters in JoogleCharts.

In [1]:
import pandas as pd
from jooglechart import JoogleChart, Filter, SeriesFilter, SuperCategoryFilter, ChartRow

# import of display functions is not needed for jooglechart
from IPython.display import display, HTML

## Series Filter
JoogleCharts has a custom filter that allows the user to filter on columns of the data, rather than rows.  The data model used in the chart must have a column of categories, followed by columns with data series, as with a line chart or column chart.  If a SeriesFilter is applied to a line chart, for example, lines will appear or disappear as the series of data are selected or deselected.

The SeriesFilter will be most effective with a chart like line chart or ***stacked*** column chart, where series can be toggled on and off without leaving gaps along the axis where the data is now "missing."

The SeriesFilter under the hood is a re-purporsed CategoryFilter, so any options CategoryFilter takes can be applied to SeriesFilter.  However, not all will be relevant; filterColumnLabel and filterColumnIndex cannot be used.

By default, all available series will be selected on rendering.  The filter can be initialized with selections by passing SelectedValues as added filter state.  The default label for the filter is "Columns," which can be overriden with the ui.label filter option.

SeriesFilter can be combined with other types of filters.  In that case, the SeriesFilter will be rendered on top.

SeriesFilter will work even when the view_cols have been set (i.e., where some columns have been hidden), and when columns have been set as roles, including tooltips.

In [2]:
bradys = ['Bobby', 'Cindy', 'Jan']
snickers = [34, 64, 24]
mounds = [20, 32, 41]
milky_way = [35, 32, 50]
payday = [23, 64, 27]

d = {}
d['Kids'] = bradys
d['Snickers'] = snickers
d['Mounds'] = mounds
d['Milky Way'] = milky_way
d['Payday'] = payday
sf_df = pd.DataFrame(d, columns=['Kids', 'Snickers', 'Mounds', 'Milky Way', 'Payday'])

chart = JoogleChart(sf_df, chart_type="ColumnChart")
chart.add_chart_options(isStacked=True, chartArea_left=40, chartArea_width=600, width=640, legend_position="top")
sf = SeriesFilter()
sf.add_options(ui_label="Candy Bars")
chart.add_filter(sf)
chart.show()

## SuperCategoryFilter  (Depracated -- use Sonar)
The SuperCategoryFilter is a category filter that can control the category filters of distinct JoogleCharts.  At times, you might have different data sets that you want to chart, but you want to control them with one filter.  The SuperCategoryFilter will allow you to do that if you have applied Category Filters to the charts.  To work predictably, the categories in the data sets should be the same.

There are several steps that must be taken to apply a SuperCategoryFilter.  First, you must instantiate the SuperCategoryFilter with a list or pandas Series of options that will appear in the filter.  Next, you must add to the SuperCategoryFilter labels for the filters that it will control.  Lastly, you must give those filters the same labels that were added to the SuperCategoryFilter.  The labels are used to create names for the filters.

Normally, you don't need to worry about the names that will be used in the javascript and html with jooglechart.  The SuperCategoryFilter is different, because the components involved will all be rendered separately, and the names must match across the rendered scripts.  Note:  **you must give the controlled filters different labels to avoid collision.**

#### Applying options and state
As a repurposed Category Filter, the SuperCategoryFilter can take the same options and state as a Category Filter.  The filterColumnIndex cannot be set, however.

It is recommended that if selectedValues are to be added to the filter states, they be added uniformly to the super filter and child filters so that display of filter and charts will be sensible when they are first rendered.

#### Showing the child filters
By default, the filters that are controlled by the super filter are hidden.  However, the controlled filters can be shown, which may be useful during development.  To show the filters, set the property **show_child_filters=True** at instantiation or later.

#### SuperCategoryFilter with SeriesFilter
Since a SeriesFilter is a Category Filter, it can also be controlled by a SuperCategoryFilter.  An importance difference is that the SuperCategoryFilter has no options selected by default, and the SeriesFilter has all options selected by default.  It is recommended that the selectedValues state property be set on the SeriesFilters and/or the super filter so that the initial rendering of the charts appears sensible.


In [3]:
# create two DataFrames
categories = ['Baltic', 'Mediterranean', 'Park Place']
hotels = [5, 7, 3]
houses = [20, 32, 41]
d1 = {}
d1['categories'] = categories
d1['hotels'] = hotels
d2 = {}
d2['categories'] = categories
d2['houses'] = houses
df1 = pd.DataFrame(d1, columns=['categories', 'hotels'])
df2 = pd.DataFrame(d2, columns=['categories', 'houses'])

# create two charts
chart1 = JoogleChart(df1)
chart2 = JoogleChart(df2)

# create two filters, giving them DIFFERENT labels; using a variable is recommended.
filter1_label = "filter1"
filter1 = Filter("CategoryFilter")
filter1.set_filter_label(filter1_label)
filter1.add_options(filterColumnIndex=0)
chart1.add_filter(filter1)

filter2_label = "filter2"
filter2 = Filter("CategoryFilter")
filter2.set_filter_label(filter2_label)
filter2.add_options(filterColumnIndex=0)
chart2.add_filter(filter2)

# create the super filter, passing the same labels used on the filters
scf = SuperCategoryFilter(df1.categories)
scf.add_filter_label(filter1_label)
scf.add_filter_label(filter2_label)

# ----------------------------------------------------------------------------------------------
# To show the child filters, either pass a parameter or set the property
# scf = SuperCategoryFilter(df1.categories, show_child_filters=True)
#      OR
# scf.show_child_filters = True
# ----------------------------------------------------------------------------------------------

# add chart options
chart1.add_chart_options(colors=['darkcyan'], chartArea_left=40, width=500, chartArea_width=350)
chart2.add_chart_options(colors=['coral'], chartArea_left=40, width=500, chartArea_width=350)

# set a common list of selectedValues, if selectedValues are desired
selected_values = ['Baltic', 'Park Place']
filter1.add_state(selectedValues=selected_values)
filter2.add_state(selectedValues=selected_values)
scf.add_state(selectedValues=selected_values)

# display all three elements, here using a common header
display(HTML("<h2>Houses and Hotels on Three Select Properties</h2></br>"))
scf.show()
row = ChartRow(chart1, chart2)
row.show()